Import dependencies

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the model
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])




Unzip file of xray and non-xray

In [ ]:
# Specify the path to your zip file
zip_file_path = '/content/non_xray.zip'

# Specify the directory where you want to extract the contents
extracted_path = '/content'

# Unzip the file
import zipfile
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# Print a message indicating successful extraction
print(f"File '{zip_file_path}' successfully extracted to '{extracted_path}'.")


File '/content/non_xray.zip' successfully extracted to '/content'.


Train classification model

In [ ]:
# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        '/content/dataset/training',  # Main directory containing subdirectories for each class
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',  # Since it's a binary classification task

        # Specify the classes
        classes=['xray', 'non_xray'],

        # Shuffle the data for better training
        shuffle=True)

# Train the model
model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples // train_generator.batch_size,
      epochs=20)

# Save the model
model.save('xray_classification_model.h5')

Found 1018 images belonging to 2 classes.
Epoch 1/20
 6/31 [====>.........................] - ETA: 1:35 - loss: 2.6677 - accuracy: 0.5156

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


31/31 [==============================] - 134s 4s/step - loss: 0.9380 - accuracy: 0.6349
Epoch 2/20
31/31 [==============================] - 124s 4s/step - loss: 0.3999 - accuracy: 0.7596
Epoch 3/20
31/31 [==============================] - 124s 4s/step - loss: 0.3781 - accuracy: 0.8590
Epoch 4/20
31/31 [==============================] - 120s 4s/step - loss: 0.3600 - accuracy: 0.8529
Epoch 5/20
31/31 [==============================] - 126s 4s/step - loss: 0.3082 - accuracy: 0.9037
Epoch 6/20
31/31 [==============================] - 130s 4s/step - loss: 0.2898 - accuracy: 0.9300
Epoch 7/20
31/31 [==============================] - 120s 4s/step - loss: 0.2717 - accuracy: 0.9493
Epoch 8/20
31/31 [==============================] - 122s 4s/step - loss: 0.2585 - accuracy: 0.9564
Epoch 9/20
31/31 [==============================] - 123s 4s/step - loss: 0.2562 - accuracy: 0.9584
Epoch 10/20
31/31 [==============================] - 132s 4s/step - loss: 0.2301 - accuracy: 0.9757
Epoch 11/20
31/31 [=

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Upload image and test the model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
import io

# Load the trained model
model = tf.keras.models.load_model('/content/compressed.h5')

# Upload an image from your device
uploaded = files.upload()

# Get the uploaded image
uploaded_file_path = list(uploaded.keys())[0]
img = image.load_img(io.BytesIO(uploaded[uploaded_file_path]), target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Rescale the pixel values to the range [0, 1]

# Make predictions
predictions = model.predict(img_array)
print(predictions)

# Interpret the predictions
if predictions[0, 0] < 0.5:
    print("Prediction: X-ray")
else:
    print("Prediction: Non-X-ray")
